In [1]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from funpymodeling.exploratory import freq_tbl
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('venecia.csv')

In [3]:
df= df.fillna(method="bfill")
df= df.fillna(method="ffill")
df = df.bfill()
df= df.ffill()

C:\Users\HP\AppData\Local\Temp\ipykernel_16752\1730909967.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method="bfill")
C:\Users\HP\AppData\Local\Temp\ipykernel_16752\1730909967.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method="ffill")


**Transformando variables a tipo dicotómicas**

In [4]:

#bathrooms_text
df["bathroomsd"] = df["bathrooms_text"].astype(str).apply(lambda x: 1 if "2" in x or "3" in x or "4" in x else 0)
#room_type Entire home
df["E_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Entire home/apt" else 0)
#room_type Private home
df["P_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Private room" else 0)
#room_type Hotel home
df["H_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Hotel room" else 0)
#instant_bookabled
df["instant_bookabled"] = df["instant_bookable"].apply(lambda x: 1 if x == "t" else 0)
#Price
df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)
Mprice = df["price"].median()
df["D_price"] = df["price"].apply(lambda x: 1 if x >= Mprice else 0)
#host_total_listings_count
Mlistings = df["host_total_listings_count"].median()
df["D_host_listings"] = df["host_total_listings_count"].apply(lambda x: 1 if x >= Mlistings else 0)
#host_total_listings_count
Maccuracy = df["host_total_listings_count"].median()
df["D_accuracy"] = df["review_scores_accuracy"].apply(lambda x: 1 if x >= Maccuracy else 0)
#availability_30
Mavailability = df["availability_30"].median()
df["D_availability"] = df["availability_30"].apply(lambda x: 1 if x >= Mavailability else 0)
#availability_90
Mavailability90 = df["availability_90"].median()
df["D_availability90"] = df["availability_90"].apply(lambda x: 1 if x >= Mavailability90 else 0)
#number_of_reviews
Mreviews = df["number_of_reviews"].median()
df["D_reviews"] = df["number_of_reviews"].apply(lambda x: 1 if x >= Mreviews else 0)
#review_scores_rating
Mrating = df["review_scores_rating"].median()
df["D_rating"] = df["review_scores_rating"].apply(lambda x: 1 if x >= Mrating else 0)
#review_scores_rating
Maccommodates = df["accommodates"].median()
df["D_accommodates"] = df["accommodates"].apply(lambda x: 1 if x >= Maccommodates else 0)



<>:12: SyntaxWarning: invalid escape sequence '\$'
<>:12: SyntaxWarning: invalid escape sequence '\$'
C:\Users\HP\AppData\Local\Temp\ipykernel_16752\2275433636.py:12: SyntaxWarning: invalid escape sequence '\$'
  df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)


In [5]:
df.head(5)

,id,scrape_id,last_scraped,source,name,description,neighborhood_overview,host_id,host_name,host_since,...,H_room_typed,instant_bookabled,D_price,D_host_listings,D_accuracy,D_availability,D_availability90,D_reviews,D_rating,D_accommodates
0,6624.0,2.025030e+13,02/03/2025,city scrape,Venice garden charme and canal view!,I'm thrilled to share my apartment with garden...,We are in the middle of a residential area cal...,15016,Francesca,27/04/2009,...,0,0,1,1,0,1,1,1,1,1
1,27116.0,2.025030e+13,02/03/2025,city scrape,Junior Suite with balcony overlooking canal,"Few steps from Saint Mark square, on a picture...","The area is very beautiful and characteristic,...",116144,GiÃ²&GiÃ²,30/04/2010,...,0,1,1,0,0,0,0,1,1,0
2,44527.0,2.025030e+13,02/03/2025,city scrape,Luxury Historic Apartment on Calm Sunny Canal,An apartment that blends Gothic style with ini...,"Cannaregio is a well-connected, truly Venetian...",120215,Marc & Elisabeth,07/05/2010,...,0,0,1,1,0,1,1,1,1,1
3,47383.0,2.025030e+13,02/03/2025,city scrape,Ca' Barba B&B: room n102 (jacuzzi shower),"Ca' Barb B&B, a few steps away from Rialto bri...",San Polo is highly rated by guests for walking...,214390,Alessandro,27/08/2010,...,1,1,0,1,0,0,1,1,1,0
4,49656.0,2.025030e+13,02/03/2025,city scrape,Ca' Foscari Apartment,Itâ€™s a lovely quiet newly restored flat with...,The apartment is on the first floor (second fl...,226598,Andrea,07/09/2010,...,0,1,0,0,0,0,0,1,1,0


**CASO 1: accommodates, bathrooms, room_type**

In [6]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_accommodates', 'bathroomsd', 'E_room_typed']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 0, 1], dtype=int64)

In [7]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[736 467]
 [344 830]]


In [8]:
#Caulculo de la precisuin del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6399383191981496


In [9]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6588136306268405


In [10]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.706984667802385


**CASO 2:** 
**price, host_listings, accuracy**

In [11]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_price', 'D_host_listings', 'D_accuracy']]
Var_Dep = df['D_availability']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [12]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[490 642]
 [490 755]]


In [13]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5404438081603435


In [14]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5237694572991165


In [15]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.606425702811245


**CASO 3:** 
**availability, number_of_reviews, review_scores_rating**

In [16]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_availability', 'D_reviews', 'D_rating']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [17]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[653 493]
 [527 704]]


In [18]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5881370091896407


In [19]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5708876735380732


In [20]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5718927701056052


**CASO 4:** 
**bathrooms, number_of_reviews, price**

In [33]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_reviews', 'D_price']]
Var_Dep = df['D_availability']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [34]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[453 702]
 [400 822]]


In [35]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5393700787401575


In [36]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5363904080774085


In [37]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6726677577741408


**CASO 5:** 
**accommodates, review_scores_rating, availability_30**

In [38]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_accommodates', 'D_reviews', 'D_availability']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [39]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[714 447]
 [376 840]]


In [40]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6526806526806527


In [41]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6537652503155238


In [42]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6907894736842105


**CASO 6:** 
**instant_bookable, review_scores_rating, availability_90**

In [31]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['instant_bookabled', 'D_reviews', 'D_availability90']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [32]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[632 556]
 [549 640]]


In [43]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6526806526806527


In [44]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6537652503155238


In [45]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6907894736842105


**CASO 7:** 
**bathrooms, availability_30, host_listings**

In [46]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_availability', 'D_host_listings']]
Var_Dep = df['D_reviews']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [47]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[663 492]
 [505 717]]


In [48]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5930521091811415


In [49]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5805637358014304


In [50]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5867430441898527


**CASO 8:** 
**price, accommodates, review_scores_rating**

In [51]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_price', 'D_accommodates', 'D_rating']]
Var_Dep = df['D_reviews']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([0, 1, 1, ..., 1, 0, 0], dtype=int64)

In [52]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[637 562]
 [549 629]]


In [53]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5281276238455079


In [54]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5326041228439209


In [55]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.533955857385399


**CASO 9:** 
**bathrooms, review_scores_accuracy, review_scores_rating**

In [56]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_accuracy', 'D_rating']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [57]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[945 246]
 [614 572]]


In [58]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6992665036674817


In [59]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.638199411022297


In [60]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.4822934232715008


**CASO 10:** 
**availabilit_90, host_total_listings_count, number_of_reviews**

In [61]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_availability90', 'D_host_listings', 'D_reviews']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [62]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[632 555]
 [554 636]]


In [63]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5340050377833753


In [64]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5334455195624737


In [65]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.534453781512605
